In [1]:
class TourInfo:
    title = None
    price = None
    comment = None
    point = None
    period1 = None
    period2 = None
    
    def __init__(self, title, price, comment, point, period1, period2):
        self.title = title; self.price = price; self.comment = comment; self.point = point
        self.period1 = period1; self.period2 = period2
        
    def __str__(self):
        return ("%s, %d, %s, %f, %s, %s"%(self.title, self.price, self.comment, self.point, self.period1, self.period2))
        

from bs4 import BeautifulSoup as bs
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
import re



#명시적 대기를 위해서
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

import time

url = 'http://tour.interpark.com/'
keyword = '미국여행'
mylist = list()

driver = wd.Chrome(executable_path="chromedriver.exe")
driver.get(url)
driver.find_element_by_id('SearchGNBText').send_keys(keyword)
driver.find_element_by_css_selector('button.search-btn').click()

#명시적 대기 : 특정 element가 load될 때까지 대기
#암시적 대기 : 일정 시간 대기

try :
    WebDriverWait(driver, 10).until(ec.presence_of_all_elements_located((By.CLASS_NAME, 'oTravelBox')))
    #지정한 요소가 로딩되면 대기 종료
    #<div class = 'oTravelBox'..>
    
except Exception as e:
    print("Exception Occurrance, reason =",e.reason)
    
    
# 암묵적 대기 # 둘중에 하나 사용

driver.implicitly_wait(10)
#time.sleep(10)
driver.find_element_by_class_name
driver.find_element_by_css_selector('.oTravelBox > .boxList >.moreBtnWrap >.moreBtn').click()
#driver.find_element_by_css_selector('button.moreBtn').click() # 위랑 동일

mylist = []
for page in range(1,2):
    # 상품명, comment, 가격, 평점, 이미지, 링크, 기간, 기간2
    # proTit, proSub, proPrice, proInfo, img, onclick의 주소, proInfo, proinfo
    time.sleep(2)
    boxItems = driver.find_elements_by_css_selector('.oTravelBox > .boxList >.boxItem')

    
  
    for box in boxItems:
        # print('Title > ',box.find_element_by_css_selector('h5.proTit').text)
        title = box.find_element_by_css_selector('h5.proTit').text
        # print('가격 > ',box.find_element_by_css_selector('strong.proPrice').text)  
        price = box.find_element_by_css_selector('strong.proPrice').text
        price = re.sub(r',',"",price.split(' ')[0])
        price = int(price)
        #print('가격 ->> ',int(price))
        #print('커멘트 >', box.find_element_by_css_selector('p.proSub').text)
        comment = box.find_element_by_css_selector('p.proSub').text
        # print('평점 >', box.find_elements_by_css_selector('p.proInfo')[2].text)   
        point = box.find_elements_by_css_selector('p.proInfo')[2].text
        point = float(point.split(' ')[1])
                       
        # print('기간1 >', box.find_elements_by_css_selector('p.proInfo')[0].text)       
        period1 = box.find_elements_by_css_selector('p.proInfo')[0].text
        period1 = period1.split(':')[1].strip()
        
        
        # print('기간2 >', box.find_elements_by_css_selector('p.proInfo')[1].text)
        period2 =  box.find_elements_by_css_selector('p.proInfo')[1].text
        period2 = period2.split(':')[1].strip()
        
        temp = TourInfo(title, price, comment, point, period1, period2)
        
        mylist.append(temp)
driver.close()

import pymongo
from pymongo import MongoClient


client = MongoClient('localhost', 27017)
db = client.test

db['TourInfo']

for a in mylist:
    db.TourInfo.insert_one({'Title':a.title, 'Price':a.price, 'Comment':a.comment, 'Point':a.point, 'Period1':a.period1, 'Period2':a.period2})

In [23]:
collection = db['TourInfo']
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test'), 'TourInfo')

In [115]:
cursor = collection.find({}, {"_id" : False})
cursor

In [116]:
import pandas as pd

In [117]:
df = pd.DataFrame(list(cursor))
df

,Title,Price,Comment,Point,Period1,Period2
0,[★LUCKY USA★] 미서부 두도시여행 샌프란시스코 & LA 5박7일,1490000,아시아나이용/자유여행/리턴연장가능,8.2,5박7일인천-샌프란시스코-로스엔젤레스-인천,2019.09.10~2019.12.23
1,[★미서부 두도시여행★] 샌프란시스코/라스베가스 자유 7일,1390000,유나이티드항공/자유여행/리턴연장가능,0.0,5박7일인천-샌프란시스코-라스베가스-샌프란시스코-인천,2019.09.13~2019.10.27
2,[미서부 10일] 세도시 샌프란시스코+라스베가스+LA 자유여행_유나이티드항공,1890000,유나이티드항공이용/자유여행/호텔8박포함,0.0,8박10일인천-샌프란시스코-라스베가스-로스엔젤레스-인천,2019.09.10~2019.10.27
3,◈내가만드는 여행◈ 뉴욕 선택과 자유 8일,2090000,나를 위한 여행,0.0,6박8일인천-뉴욕-인천,2019.09.15~2020.03.29
4,◈내가만드는 여행◈ 뉴욕 선택과 자유 10일_저녁출발 아시아나항공,2390000,선택과 집중,0.0,7박10일인천-뉴욕-인천,2019.11.27~2020.03.25
5,◈내가만드는 여행◈ 뉴욕 자세히보기 7일,2090000,나를 위한 여행,0.0,5박7일인천-뉴욕-인천,2019.11.25~2020.03.30
6,◈내가만드는 여행/맨하탄 3박◈ 뉴욕 선택과 자유 8일,2499000,나를 위한 여행,0.0,6박8일인천-뉴욕-인천,2019.09.15~2020.03.29
7,■미대륙 완전일주■미국/캐나다 동부+미국 서부 3대도시&4대캐년 16일,2999000,세계 3대폭포 중 하나인 나이아가라&4대캐년을 한번에,0.0,14박16일인천-로스앤젤레스-바스토우-라스베가스-3대캐년-라플린-로스앤젤레스-뉴욕-...,2019.09.16~2019.10.21
8,[KE 보스턴 취항기념] 뉴욕/보스턴 7일 (뉴욕 당일투어포함),2490000,보스턴 취항기념 두도시여행,0.0,5박7일인천-뉴욕-보스턴-인천,2019.09.10~2019.10.31
9,[3대캐년&홀스슈밴드 투어포함] 라스베가스 자유 6일,1590000,3대캐년&홀스슈밴드 당일투어가 포함된 라스베가스 자유6일!,9.8,4박6일인천-샌프란시스코-라스베가스-샌프란시스코-인천,2019.09.10~2019.12.23


In [98]:
keylist =[]
total =[]


for i, cur in enumerate(cursor):
    vallist = []
    for key, val in cur.items():
        if i == 0:
            keylist.append(key)
        vallist.append(val)
    total.append(vallist)

In [99]:
keylist

['Title', 'Price', 'Comment', 'Point', 'Period1', 'Period2']

In [101]:
total

[['[★LUCKY USA★] 미서부 두도시여행 샌프란시스코 & LA 5박7일',
  1490000,
  '아시아나이용/자유여행/리턴연장가능',
  8.2,
  '5박7일인천-샌프란시스코-로스엔젤레스-인천',
  '2019.09.10~2019.12.23'],
 ['[★미서부 두도시여행★] 샌프란시스코/라스베가스 자유 7일',
  1390000,
  '유나이티드항공/자유여행/리턴연장가능',
  0.0,
  '5박7일인천-샌프란시스코-라스베가스-샌프란시스코-인천',
  '2019.09.13~2019.10.27'],
 ['[미서부 10일] 세도시 샌프란시스코+라스베가스+LA 자유여행_유나이티드항공',
  1890000,
  '유나이티드항공이용/자유여행/호텔8박포함',
  0.0,
  '8박10일인천-샌프란시스코-라스베가스-로스엔젤레스-인천',
  '2019.09.10~2019.10.27'],
 ['◈내가만드는 여행◈ 뉴욕 선택과 자유 8일',
  2090000,
  '나를 위한 여행',
  0.0,
  '6박8일인천-뉴욕-인천',
  '2019.09.15~2020.03.29'],
 ['◈내가만드는 여행◈ 뉴욕 선택과 자유 10일_저녁출발 아시아나항공',
  2390000,
  '선택과 집중',
  0.0,
  '7박10일인천-뉴욕-인천',
  '2019.11.27~2020.03.25'],
 ['◈내가만드는 여행◈ 뉴욕 자세히보기 7일',
  2090000,
  '나를 위한 여행',
  0.0,
  '5박7일인천-뉴욕-인천',
  '2019.11.25~2020.03.30'],
 ['◈내가만드는 여행/맨하탄 3박◈ 뉴욕 선택과 자유 8일',
  2499000,
  '나를 위한 여행',
  0.0,
  '6박8일인천-뉴욕-인천',
  '2019.09.15~2020.03.29'],
 ['■미대륙 완전일주■미국/캐나다 동부+미국 서부 3대도시&4대캐년 16일',
  2999000,
  '세계 3대폭포 중 하나인 나이아가라&4대캐년을 한번에',
  0.0,
  '14박16일인

In [110]:
tit = []
pri = []
com = []
poi = []
pe1 = []
pe2 = []

for n in total:
    tit.append(n[0])
    pri.append(n[1])
    com.append(n[2])
    poi.append(n[3])
    pe1.append(n[4])
    pe2.append(n[5])
lt = [tit, pri, com, poi, pe1, pe2]

In [111]:
dic = {}
for col, val in zip(keylist, lt):
    dic[col] = val

In [112]:
df = pd.DataFrame(dic)

In [113]:
df

,Title,Price,Comment,Point,Period1,Period2
0,[★LUCKY USA★] 미서부 두도시여행 샌프란시스코 & LA 5박7일,1490000,아시아나이용/자유여행/리턴연장가능,8.2,5박7일인천-샌프란시스코-로스엔젤레스-인천,2019.09.10~2019.12.23
1,[★미서부 두도시여행★] 샌프란시스코/라스베가스 자유 7일,1390000,유나이티드항공/자유여행/리턴연장가능,0.0,5박7일인천-샌프란시스코-라스베가스-샌프란시스코-인천,2019.09.13~2019.10.27
2,[미서부 10일] 세도시 샌프란시스코+라스베가스+LA 자유여행_유나이티드항공,1890000,유나이티드항공이용/자유여행/호텔8박포함,0.0,8박10일인천-샌프란시스코-라스베가스-로스엔젤레스-인천,2019.09.10~2019.10.27
3,◈내가만드는 여행◈ 뉴욕 선택과 자유 8일,2090000,나를 위한 여행,0.0,6박8일인천-뉴욕-인천,2019.09.15~2020.03.29
4,◈내가만드는 여행◈ 뉴욕 선택과 자유 10일_저녁출발 아시아나항공,2390000,선택과 집중,0.0,7박10일인천-뉴욕-인천,2019.11.27~2020.03.25
5,◈내가만드는 여행◈ 뉴욕 자세히보기 7일,2090000,나를 위한 여행,0.0,5박7일인천-뉴욕-인천,2019.11.25~2020.03.30
6,◈내가만드는 여행/맨하탄 3박◈ 뉴욕 선택과 자유 8일,2499000,나를 위한 여행,0.0,6박8일인천-뉴욕-인천,2019.09.15~2020.03.29
7,■미대륙 완전일주■미국/캐나다 동부+미국 서부 3대도시&4대캐년 16일,2999000,세계 3대폭포 중 하나인 나이아가라&4대캐년을 한번에,0.0,14박16일인천-로스앤젤레스-바스토우-라스베가스-3대캐년-라플린-로스앤젤레스-뉴욕-...,2019.09.16~2019.10.21
8,[KE 보스턴 취항기념] 뉴욕/보스턴 7일 (뉴욕 당일투어포함),2490000,보스턴 취항기념 두도시여행,0.0,5박7일인천-뉴욕-보스턴-인천,2019.09.10~2019.10.31
9,[3대캐년&홀스슈밴드 투어포함] 라스베가스 자유 6일,1590000,3대캐년&홀스슈밴드 당일투어가 포함된 라스베가스 자유6일!,9.8,4박6일인천-샌프란시스코-라스베가스-샌프란시스코-인천,2019.09.10~2019.12.23


for i, docu in enumerate(cursor):
    rowcon = ''
    keycon = ''
    #print(docu)
    for key, val in docu.items():
        rowcon = rowcon + '\t' + str(val)[:3]
        keycon = keycon + '\t' + str(key)
    if(i == 0):
        print(keycon)
        print(rowcon)
    else:
        print(rowcon)
    
        